In [1]:
%reload_ext dotenv
%dotenv .env

In [2]:
import os

from mechanicalsoup import StatefulBrowser

browser = StatefulBrowser()

#### Get all tram line URLs

In [3]:
browser.open(os.environ.get('TIMETABLE_URL'))

<Response [200]>

In [4]:
tram_lines = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("a", class_=lambda x: x and x.startswith("linia"))
    if len(item.text.strip()) <= 2
}

tram_lines

{'1': '?lang=PL&linia=1',
 '3': '?lang=PL&linia=3',
 '4': '?lang=PL&linia=4',
 '5': '?lang=PL&linia=5',
 '8': '?lang=PL&linia=8',
 '9': '?lang=PL&linia=9',
 '10': '?lang=PL&linia=10',
 '11': '?lang=PL&linia=11',
 '13': '?lang=PL&linia=13',
 '14': '?lang=PL&linia=14',
 '16': '?lang=PL&linia=16',
 '17': '?lang=PL&linia=17',
 '18': '?lang=PL&linia=18',
 '20': '?lang=PL&linia=20',
 '21': '?lang=PL&linia=21',
 '22': '?lang=PL&linia=22',
 '24': '?lang=PL&linia=24',
 '50': '?lang=PL&linia=50',
 '52': '?lang=PL&linia=52',
 '62': '?lang=PL&linia=62',
 '64': '?lang=PL&linia=64',
 '69': '?lang=PL&linia=69',
 '72': '?lang=PL&linia=72'}

#### Get line no. 1 variants

In [5]:
browser.open(os.environ.get('TIMETABLE_URL') + list(tram_lines.values())[0])

<Response [200]>

In [6]:
variant_urls = {
    item.text.strip(): item.get("href")[1:]
    for item in browser.page.findAll("table")[7].findAll("a")
}

variant_urls

{'Rondo Hipokratesa - Cichy Kącik': '?lang=PL&akcja=index&rozklad=20240503&linia=1__1',
 'Cichy Kącik - Rondo Hipokratesa': '?lang=PL&akcja=index&rozklad=20240503&linia=1__2'}

#### Get first variant's stops

In [7]:
browser.open(os.environ.get('TIMETABLE_URL') + list(variant_urls.values())[0])

<Response [200]>

In [8]:
stop_urls = {
    item.text.strip(): item.find("a") and item.find("a").get("href")[1:]
    for item in browser.page.findAll("table")[10].findAll("td")
    if item.text.strip()
}

stop_urls

{'Rondo Hipokratesa 02': '?lang=PL&rozklad=20240503&linia=1__1__1',
 'DH Wanda 02': '?lang=PL&rozklad=20240503&linia=1__1__2',
 'Rondo Kocmyrzowskie im. Ks. Gorzelanego 02': '?lang=PL&rozklad=20240503&linia=1__1__3',
 'Bieńczycka 01': '?lang=PL&rozklad=20240503&linia=1__1__4',
 'Rondo Czyżyńskie 04': '?lang=PL&rozklad=20240503&linia=1__1__5',
 'Centralna 02': '?lang=PL&rozklad=20240503&linia=1__1__6',
 'Rondo 308. Dywizjonu 02': '?lang=PL&rozklad=20240503&linia=1__1__7',
 'Ogród Doświadczeń 02': '?lang=PL&rozklad=20240503&linia=1__1__8',
 'TAURON Arena Kraków al. Pokoju 02': '?lang=PL&rozklad=20240503&linia=1__1__9',
 'Dąbie 02': '?lang=PL&rozklad=20240503&linia=1__1__10',
 'Ofiar Dąbia 02': '?lang=PL&rozklad=20240503&linia=1__1__11',
 'Fabryczna 02': '?lang=PL&rozklad=20240503&linia=1__1__12',
 'Francesco Nullo 02': '?lang=PL&rozklad=20240503&linia=1__1__13',
 'Teatr Variété 02': '?lang=PL&rozklad=20240503&linia=1__1__14',
 'Rondo Grzegórzeckie 02': '?lang=PL&rozklad=20240503&linia=1_

#### Get first stop's departures

In [9]:
browser.open(os.environ.get('TIMETABLE_URL') + list(stop_urls.values())[0])

<Response [200]>

In [10]:
for item in browser.page.findAll("table")[13].findAll("tr")[1:-2]:
    cells = item.findAll("td")
    
    hour = int(cells[0].text)
    weekday_minutes = [int(minute) for minute in cells[1].text.strip().split()]
    saturday_minutes = [int(minute) for minute in cells[2].text.strip().split()]
    sunday_minutes = [int(minute) for minute in cells[3].text.strip().split()]
    
    print(hour, weekday_minutes, saturday_minutes, sunday_minutes, sep="\t")

3	[]	[]	[]
4	[48]	[54]	[]
5	[1, 15, 32, 47]	[14, 34, 54]	[1, 31]
6	[3, 17, 33, 47]	[14, 34, 54]	[1, 31]
7	[3, 18, 33, 48]	[14, 34, 54]	[1, 31]
8	[3, 18, 33, 48]	[13, 33, 53]	[3, 33]
9	[3, 18, 33, 48]	[13, 33, 53]	[3, 33, 53]
10	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
11	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
12	[3, 18, 33, 48]	[13, 33, 53]	[13, 33, 53]
13	[3, 16, 32, 47]	[13, 33, 53]	[13, 33, 53]
14	[4, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
15	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
16	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
17	[6, 21, 36, 51]	[13, 33, 53]	[13, 33, 53]
18	[6, 21, 36, 53]	[13, 33, 53]	[13, 33, 53]
19	[10, 25, 40, 57]	[13, 33, 53]	[13, 33, 53]
20	[10, 23, 38, 53]	[14, 34, 54]	[14, 34, 54]
21	[12, 32, 51]	[14, 34, 54]	[14, 34, 54]
22	[13, 33, 54]	[14, 34]	[14, 34]
23	[]	[4]	[4]
0	[]	[]	[]
